# OpenSearch Warming Up (기존 인덱스를 한글 형태소 분석기 사용하여 리인덱식)
>이 노트북은 SageMaker Studio* Data Science 3.0 kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.



여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)


# 0. 필요 패키지 설치

In [44]:
# install_needed = True  # should only be True once
install_needed = False  # should only be True once

In [45]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install opensearch-py==2.3.1
    !{sys.executable} -m pip install langchain==0.0.302    
    !{sys.executable} -m pip install sqlalchemy==2.0.1
    
    IPython.Application.instance().kernel.do_shutdown(True)    
    

# 1. 환경 세팅

In [62]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [63]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Embedding 모델 선택

In [64]:
from utils.rag import get_embedding_model
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [65]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [50]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 2. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [51]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [52]:
os.makedirs("data", exist_ok=True)

In [53]:
def preprocess_data(df):

    ldf = df.copy()
    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [54]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [55]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)
import time
documents_fsi = loader.load()


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [56]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):
        
        #print ("previous:", doc)
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        doc.metadata['timestamp'] = time.time()
        
        #print ("new:", doc)
        #print ("==")

        
create_metadata(documents_fsi)

In [57]:
print (len(documents_fsi))
print (documents_fsi[0])

92
page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n' metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1696469021.6739798}


## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. <BR>
또한 임베딩 모델에는 입력 토큰 길이는 KoSimCSERobert는 512개, titanEmbedding(8,912개) 토큰으로 제한되어 있습니다. <BR>
이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 500자가 겹치는 약 92자의 청크를 생성합니다.

In [58]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 50
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


# 3. OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
### 아래 셀에 다음의 정보가 입력이 되어야 합니다.
```
opensearch_domain_endpoint = "<Type Domain Endpoint>"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

```


In [59]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index
from utils.rag import create_index, add_doc, search_document

In [60]:
aws_region = 'us-east-1'

os.environ["OpenSearch_UserName"] = "<Type UserName>"
os.environ["OpenSearch_UserPassword"] = "<Type Password>"

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

opensearch_domain_endpoint = "<Type your domain endpoint>"
#opensearch_domain_endpoint = "https://search-genai-demo-domain-36etfsphifulgs7ckovsavchca.us-east-1.es.amazonaws.com"

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

aws_client = create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [61]:
index_name = "genai-demo-index-v1"
index_exists = check_if_index_exists(aws_client, index_name)

if index_exists:
    delete_index(aws_client, index_name)
else:
    print("Index does not exist")

ConnectionError: ConnectionError(HTTPSConnectionPool(host='%3ctype%20your%20domain%20endpoint%3e', port=443): Max retries exceeded with url: /genai-demo-index-v1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f63d917cd30>: Failed to establish a new connection: [Errno -2] Name or service not known'))) caused by: ConnectionError(HTTPSConnectionPool(host='%3ctype%20your%20domain%20endpoint%3e', port=443): Max retries exceeded with url: /genai-demo-index-v1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f63d917cd30>: Failed to establish a new connection: [Errno -2] Name or service not known')))

## 인덱스 생성

In [23]:
%%time

# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
from langchain.vectorstores import OpenSearchVectorSearch
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

CPU times: user 215 ms, sys: 7.62 ms, total: 223 ms
Wall time: 9.71 s


## 인덱스 확인

In [24]:
from pprint import pprint
from utils.rag import to_pickle

In [25]:
index_info = aws_client.indices.get(index=index_name)
pprint (index_info)
to_pickle(index_info, "../utils/tokenizer-index-info-template.pkl")

{'genai-demo-index-v1': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                 'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                   'type': 'keyword'}},
                                                                                            'type': 'text'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                 'type': 'keyword'}},
                                                                                          't

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#

### 인덱싱 수정하기 (형태소 분석시 사용 enablement)

In [26]:
new_index_name = f'{index_name}-with-tokenizer'
new_index_name

'genai-demo-index-v1-with-tokenizer'

In [27]:
## Setting for "은전한잎" Tokenizer (변경 없음)
index_info[index_name]["settings"]["analysis"] = {
    "tokenizer": {
        "seunjeon": {
            "type": "seunjeon_tokenizer"
        }
    },
    "analyzer": {
        "my_analyzer": {
            "type": "custom",
            "tokenizer": "seunjeon"
        }
    }
}

## Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

## Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}

del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]

In [28]:
pprint (new_index_info)

{'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                         'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                           'type': 'keyword'}},
                                                                    'type': 'text'},
                                                         'timestamp': {'type': 'float'},
                                                         'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'}}},
                             'text': {'analyzer': 'my_analyzer',
                                      'fields': {'keyword': {'ignore_above': 256,
                                                             'type':

In [29]:
index_info = aws_client.indices.get(index=new_index_name)
pprint (index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                  'type': 'keyword'}},
                                                                                                           'type': 'text'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                         

### 형태소 분석기용 인덱서 생성

In [30]:
index_exists = check_if_index_exists(aws_client, new_index_name)
if index_exists: delete_index(aws_client, new_index_name)
else: print("Index does not exist")

index_name=genai-demo-index-v1-with-tokenizer, exists=True

Deleting index:
{'acknowledged': True}


In [31]:
create_index(
    aws_client,
    index_name=new_index_name,
    index_body=new_index_info
)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-demo-index-v1-with-tokenizer'}


### Re-indexing

In [32]:
_reindex = {
    "source": {"index": index_name},
    "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

_reindex: 
 {'source': {'index': 'genai-demo-index-v1'}, 'dest': {'index': 'genai-demo-index-v1-with-tokenizer'}}


In [33]:
aws_client.reindex(_reindex)

{'took': 334,
 'timed_out': False,
 'total': 92,
 'updated': 0,
 'created': 92,
 'deleted': 0,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

# 6. 키워드 검색

### 'Text" 에 "약관, 뱅킹" 단어를 검색합니다.

In [34]:
from utils.rag import generate_opensearch_AndQuery, parse_keyword_response

In [35]:
def run_keyword_query(q, aws_client, index_name, min_shoud_match=0):

    query = {
        "query": {
            "match": {
                "text": {
                    "query": f"{q}",
                    "minimum_should_match": f'{min_shoud_match}%'
                }
            }
        }
    }    
    
    print("query: ", query)
    response = search_document(aws_client, query, index_name)
    
    return response

* without tokenizer

In [36]:
q = "뱅킹"
response = run_keyword_query(q, aws_client, index_name)
parse_keyword_response(response, show_size=3)

query:  {'query': {'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%'}}}}

Search results:
There is no response


* with tokenizer

In [37]:
q = "뱅킹"
response = run_keyword_query(q, aws_client, new_index_name)
parse_keyword_response(response, show_size=3)

query:  {'query': {'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%'}}}}

Search results:
# of searched docs:  10
# of display: 3
---------------------
_id in index:  745f8edf-d51e-4fe9-93b2-1f512e302707
1.3011781
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는 연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→ 장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다. 쏠(SOL)의 경로는 위와 동일하며, 비대면 실명인증후 해제 가능합니다. 비대면실명인증 - 만 14세이상 가능 - 신분증확인 (주민등록증 또는 운전면허증 또는 여권) - 본인확인 (본인명의 계좌인증 또는 영상통화) 진행 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다
---------------------
_id in index:  a373a726-25d4-465e-b42d-f8170877d1b2
1.2062757
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규

## 형태소 분석 결과 확인
"약관" 또는 "뱅킹" 확인 <BR>
* doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [38]:
doc_id = "da2d82bb-b68f-4269-b9c9-2d7dcce89a56" 

* without tokenizer

In [39]:
aws_client.termvectors(index=index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1',
 '_id': 'da2d82bb-b68f-4269-b9c9-2d7dcce89a56',
 '_version': 0,
 'found': False,
 'took': 0}

* with tokenizer

In [40]:
aws_client.termvectors(index=new_index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1-with-tokenizer',
 '_id': 'da2d82bb-b68f-4269-b9c9-2d7dcce89a56',
 '_version': 0,
 'found': False,
 'took': 0}

## Minimum_should_match 활용
- An optional parameter of type string that represents the minimum number of matching clauses for a document to be returned. This should only be used with “OR” operator, and is more flexible than a simple “and/or,” since users can set rules depending on the length of the phrase to be matched. 
    - **query에 있는 단어의 n%이상 존재하는 문서만 가져온다**
    - 75% (eg. 3 out of 4 words to be matched, or 6 out of 8)
    - 2  (minimum 2 words to be matched, irrespective of length of string)

min_shoud_match를 25, 75로 변경해 보자
 - 숫자가 작아질 수록 검색 문서가 많아진다
 - 추가정보는 아래를 참고
     - [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
     - OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
         - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)

In [41]:
q = "인터넷 뱅킹으로 예적금 해약"
response = run_keyword_query(q, aws_client, new_index_name, min_shoud_match=75)
parse_keyword_response(response, show_size=10)

query:  {'query': {'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '75%'}}}}

Search results:
# of searched docs:  2
# of display: 10
---------------------
_id in index:  1a1bd078-1838-40cb-99bb-a2320ff97e64
7.919839
ask: 인터넷 예적금 해약하려면 어떻게 해야 하나요?
Information: 인터넷에서 신규하셨고, 이후 통장발급을 받지 않으셨다면 인터넷뱅킹(http://bank.shinhan.com)의 금융상품 예금/신탁 해지 메뉴를 통해 해지하실 수 있습니다.
---------------------
_id in index:  a373a726-25d4-465e-b42d-f8170877d1b2
5.7616262
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
---------------------


## index example (참고용)

In [42]:
test_index_info = {
    "settings": {
        "index": {
            "number_of_shards": "5",
            "knn.algo_param": {"ef_search": "512"},
            "knn": "true",
            "number_of_replicas": "2"
        },
        "analysis": {
            "tokenizer": {
                "seunjeon": {
                    "type": "seunjeon_tokenizer"
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "type": "custom",
                    "tokenizer": "seunjeon"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "metadata": {
                "properties": {
                    "row": {"type": "long"},
                    "source": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "timestamp": {"type": "float"},
                    "type": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    }
                }
            },
            "text": {
                "type": "text",
                "analyzer": "my_analyzer",
                "search_analyzer": "my_analyzer",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "space_type": "l2",
                    "name": "hnsw",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    }
                }
            }
        }
    }
}